Helper functions copied from class hw.  Make sure you copy the chrome driver into this directory!

In [233]:
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
import re 
import urllib

#create a webdriver object and set options for headless browsing
options = Options()
options.headless = True
browser = webdriver.Chrome('./chromedriver',options=options)


#uses webdriver object to execute javascript code and get dynamically loaded webcontent
def get_js_soup(url,browser):
    browser.get(url)
    res_html = browser.execute_script('return document.body.innerHTML')
    soup = BeautifulSoup(res_html,'html.parser') #beautiful soup object to be used for parsing html content
    return soup

#tidies extracted text 
def process_bio(bio):
    bio = bio.encode('ascii',errors='ignore').decode('utf-8')       #removes non-ascii characters
    bio = re.sub('\s+',' ',bio)       #repalces repeated whitespace characters with single space
    return bio

''' More tidying
Sometimes the text extracted HTML webpage may contain javascript code and some style elements. 
This function removes script and style tags from HTML so that extracted text does not contain them.
'''
def remove_script(soup):
    for script in soup(["script", "style"]):
        script.decompose()
    return soup

#helper function to write lists to files
def write_lst(lst,file_):
    with open(file_,'w') as f:
        for l in lst:
            f.write(l)
            f.write('\n')

Function to scrape links from an indeed search result

In [234]:
def scrape_search_result_page(dir_url,page_result,browser):
    print ('-'*20,'Scraping indeed search result page '+ str(page_result)+'','-'*20)
    indeed_links = []
    #execute js on webpage to load faculty listings on webpage and get ready to parse the loaded HTML 
    soup = get_js_soup(dir_url,browser) 
    for link_holder in soup.find_all('div',class_='title'): #get list of all <div> of class 'photo nocaption'
        rel_link = link_holder.find('a')['href'] #get url
        #url returned is relative, so we need to add base url
        if rel_link != '':
            indeed_links.append('https://www.indeed.com' + rel_link) 
    print ('-'*20,'Found {} indeed search urls'.format(len(indeed_links)),'-'*20)
    return indeed_links


Build query URL string and call the scraper function and loop over n number of pages

In [235]:
start = 0 #pagnigation variable, page 1 = 0, page 2 = 10, page 3 = 30, etc
q = 'python developer' #job query string
l = 'New+York+State' #location of job
numPage = 20 #num pages to scrap links from
allLinks = [] # list to capture

for page_result in range(numPage):
    start = page_result* 10 #increment the variable used to denote the next page
    search_result_url = 'https://www.indeed.com/jobs?q='+ q +'&l='+ l +'&start='+str(start) #build query string
    print(search_result_url)
    jobSearchResult = scrape_search_result_page(search_result_url,page_result, browser) # call scraper function
    allLinks.extend(jobSearchResult) #add to link
    

https://www.indeed.com/jobs?q=python developer&l=New+York+State&start=0
-------------------- Scraping indeed search result page 0 --------------------
-------------------- Found 19 indeed search urls --------------------
https://www.indeed.com/jobs?q=python developer&l=New+York+State&start=10
-------------------- Scraping indeed search result page 1 --------------------
-------------------- Found 19 indeed search urls --------------------
https://www.indeed.com/jobs?q=python developer&l=New+York+State&start=20
-------------------- Scraping indeed search result page 2 --------------------
-------------------- Found 19 indeed search urls --------------------
https://www.indeed.com/jobs?q=python developer&l=New+York+State&start=30
-------------------- Scraping indeed search result page 3 --------------------
-------------------- Found 19 indeed search urls --------------------
https://www.indeed.com/jobs?q=python developer&l=New+York+State&start=40
-------------------- Scraping indeed sea

Print links and write to file for debug

In [236]:
#Remove Duplicates
print(len(allLinks))
allLinks = list(set(allLinks))
print (len(allLinks))

376
376


In [237]:
print(allLinks)

job_urls_file = 'jobSearchResult' +q+'.txt'
write_lst(allLinks,job_urls_file)

['https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CEi_iD1h0QD-Oh4sSrQYRGe70UNTrrByMkq2RRcHzje4XUK-7OD9UueuijoYLffwwMh_KO5QDHI-C5aKS_KYMZSJfjk0alTVNcje0swgLe1X6t-9yxasZI9fhMpBub4PzlWlug4ZQcMhBbFeYJusNnbhaWv-ROkZCNcHMTVXP9g8LphlkKdT5nqE1X0fCAzUQEiPv50fVTsAAYoM72_MwhbWUfiMBj3d0FpsPmzTOs5na5ScreNUm2dMiDp6NKIgZZuVPvSC3ocOmnGr0V6ui462J5aGh1v2TgL6vCqHq6qhTLCEpOGhakIeozScxLMeKvauODQCcVjb3ViNP9QhO12GpjTgCbE-PWL_fX8VVfFo4M_HUAkbCuiKHI1X6u4e26bY6fNpYgJWyGsEdw3RnfbuQdrM8XhsAp1x_bOZOTYPYZl94xr3FWZFrQ7SSDKkVb2KLG_s-eAA==&p=13&fvj=1&vjs=3', 'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0DbX0ay-mB7EgYPm6iOF_yQM0Yq99jUNqiiBGdomfxBgUXcs5GSy0oXVzdXMorigzR149EPlKmTA6j38wLStaG39zqi4Jr__PczDHwRFaWNHXe62uoiwp4ZfhM-CZgWRMJDI7bXeuNN9iD7bejLwVj0y3AdHITXN1yc-DyOejXCiu5ImejSeOAEUqotvzypceEvCZJ0Zkd6Fd2YWHqP1n1k9InH3a8IQyj7jN9777SENf_eie9UERsJGccRIhdCvwWrZQRBBuZ-a6m-Jw7-RXULDdaTQGtnIcz8Wii8CS8OJOLUQZvBnF5hIWwfjd9sB9Dh5B4fzr-toaZER6aZb4EIWVD3sHEVwGbL2KFPkDnEV5WYR9G4aoSisFDg-nsB9CSY3uwqlC6RLT-ua4hdTuxOwbLT5

Scrape Page Data for each Link

In [239]:
from gensim.parsing.preprocessing import remove_stopwords
homepage_found = False
page_data = ''
page_data_list = []
for indeed_url in allLinks:
    try:
        page_soup = remove_script(get_js_soup(indeed_url,browser)) 
    except:
        print ('Could not access {}'.format(indeed_url))
     
    page_data = process_bio(page_soup.get_text(separator=' '))  #helper function from bio hw to clean up text
    
    #remove header
    page_data = page_data[189:] #the 189 slice removes the header of the indeed pages
    
    #remove footer
    footer_position = page_data.find('save job') #find the position of 'save job' which starts the footer
    trimStringBy = footer_position - len(page_data) #returns a negative number to trim the string by
    page_data = page_data[:trimStringBy] #drop footer
    page_data = remove_stopwords(page_data)
    page_data_list.append(page_data)
    

Print page data and write to file for debug

**Footer still has some text at the end which isn't properly cleaned

In [240]:
print(page_data_list[1])
document_set = page_data_list
page_data_file = 'pageText' +q+'.txt'
write_lst(page_data_list,page_data_file)

QlikView Developer Building Service 32BJ Benefit Funds 42 reviews New York, NY 10011 Position Summary: The candidate proven track record Qlikview end-to-end BI implementation. The candidate directly responsible development maintenance Qlikview applications. The candidate proficient Qlikview data modelling, UI design, application architecture knowledge Performance tuning dashboard. They able work agile-like development environment minimum supervision. An ideal candidate position needs proactive, self-motivated proven team-player. They collaborative team-work approach, good work ethics strong communication skills ensure client requirements correctly translated design business partners comfortable work directly. They confident skills able help guide users away bad designs. Essential Duties Responsibilities : Participate development lifecycle design, development, testing production implementation following agile methodologies Extract, transform load data multiple sources QlikView Modify ex

Summarization Using Text Rank

In [241]:
# Create single document by concatonating all documents
all_documents = ""

for doc in page_data_list:
    all_documents += doc


In [242]:
#keywords
from gensim.summarization import keywords
keywords(all_documents).split('\n')


['developer',
 'developers',
 'developing',
 'develops',
 'work',
 'working',
 'works',
 'worked',
 'experiments',
 'experimenting',
 'experiment',
 'data',
 'teams',
 'teaming',
 'new',
 'news',
 'develop functionality',
 'technologies',
 'technological',
 'requires',
 'required',
 'requirement',
 'requiring',
 'includes',
 'include',
 'engineer',
 'engineers',
 'engine',
 'designed',
 'designs',
 'designers',
 'designing',
 'designer',
 'designated',
 'applications',
 'applicant',
 'software',
 'businesses',
 'busy',
 'engineering knowledge',
 'production',
 'product',
 'productivity',
 'products',
 'productive',
 'presentations provide training developments',
 'developed insight enable',
 'python',
 'building',
 'builds',
 'buildings',
 'projects',
 'company',
 'companys',
 'support',
 'supportive',
 'supports',
 'supported',
 'supportability',
 'skill',
 'design application architecture',
 'create experiences app web',
 'directly responsible development maintenance',
 'management',

In [243]:
from gensim.summarization.summarizer import summarize
print(summarize(all_documents, word_count  = 250))

Role Responsibilities Develop reports SQL, Excel, R/Python, Tableau Automation processes Python, R, Microsoft Flow, batch scripts Develop statistically sound research papers population health studies Develop data warehouse (ETL), Data Mining, Business Intelligence (BI) Participate RFP (request proposal) evaluation vendors Deployment enterprise wide technology solutions Work project managers develop, refine, test workflows Foster relationships trust business partners Miscellaneous data/IT tasks Qualifications Education Requirements 2-5 years work experience Must healthcare experience (1-5 years) Must database experience writing queries (1-5 years) Knowledge clinical coding systems (ICD-10, CPT, HCPCS) Working knowledge SDLC process Ability offer innovative technical solutions streamline workflows processes Process engineering mindset providing scalable, sustainable well-documented solutions Possess undergraduate degree science, engineering, applied mathematics Detail oriented ability ma

In [244]:
from gensim.summarization import mz_keywords
print(mz_keywords(all_documents,scores=True,threshold=0.001))


C:\Users\doug\Anaconda3\lib\site-packages\gensim\summarization\mz_entropy.py:65: RuntimeWarning: divide by zero encountered in log2
  log_p = np.log2(p)
C:\Users\doug\Anaconda3\lib\site-packages\gensim\summarization\mz_entropy.py:66: RuntimeWarning: invalid value encountered in multiply
  h = np.nan_to_num(p * log_p).sum(axis=0)


[('qlikview', 0.00477028639745838), ('data', 0.0037821583026482358), ('blinddata', 0.003213698247744926), ('we', 0.002866397207152274), ('seen', 0.002279169706173713), ('ibm', 0.0021424535827076147), ('companies', 0.0021245024005486822), ('trading', 0.0018915096455852534), ('google', 0.0018799485646311636), ('you', 0.0018197834796990745), ('triplebyte', 0.0018177334081779881), ('code', 0.0017316550643536475), ('hadoop', 0.0016351662085083006), ('experience', 0.0014510176021065252), ('plus', 0.001436208159495891), ('company', 0.0014316647450691006), ('get', 0.0014259181599913717), ('test', 0.0013302386027653275), ('rules', 0.0013114697307233492), ('talent', 0.0013048302032997935), ('pyspark', 0.0012717435071853344), ('software', 0.0012351205064925648), ('business', 0.001226263600548702), ('mount', 0.0011955870199740239), ('build', 0.0011547092405539033), ('sinai', 0.0011518077749310073), ('development', 0.0011501769107334365), ('contract', 0.0011501021987810646), ('spark', 0.00112390541

LDA

In [245]:
docs = page_data_list

# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

must download wordnet!!!

In [246]:
#import nltk
#nltk.download('wordnet')


In [247]:
# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

In [248]:
from nltk.corpus import stopwords



In [249]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=10)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [292]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.75)

In [293]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [294]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 1473
Number of documents: 376


In [295]:
from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [296]:
top_topics = model.top_topics(corpus, topn=10) #, num_words=10)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -0.8491.
[([(0.01685106, 'blinddata'),
   (0.015008503, 'company'),
   (0.012248428, 'test'),
   (0.010604159, 'challenge'),
   (0.010048239, 'talent'),
   (0.009737335, 'software'),
   (0.008200688, 'location'),
   (0.008075379, 'role'),
   (0.007986966, 'help'),
   (0.007980428, 'engineer')],
  -0.35065598135233733),
 ([(0.02689469, 'machine_learning'),
   (0.026276302, 'machine'),
   (0.021702606, 'learning'),
   (0.021111108, 'you'),
   (0.020721724, 'we'),
   (0.01477356, 'data'),
   (0.014265509, 'problem'),
   (0.013326308, 'knowledge'),
   (0.012268698, 'company'),
   (0.011081269, 'required')],
  -0.5585324933623045),
 ([(0.02894372, 'contract'),
   (0.024067733, 'required'),
   (0.021020595, 'python_developer'),
   (0.020459304, 'year_required'),
   (0.02024172, 'framework'),
   (0.016983405, 'hour'),
   (0.015363959, 'django'),
   (0.015187777, 'location'),
   (0.0144764995, 'this'),
   (0.014443344, 'working')],
  -0.614811110822034),
 ([(0.01451860